In [1]:
# !pip install -r requirements.txt
# langgraph
# langchain-community
# langchain-core
# langchain-openai
# langchain
# langgraph-sdk
# langgraph-checkpoint-sqlite
# langsmith
# langchainhub==0.1.15
# langchain-openai==0.1.3
# langchain==0.1.16
# langchain-core==0.1.42
# pygraphviz==1.12
# llama-index
# fastapi
# uvicorn

!pip install -qU langgraph langchain-community langchain-core langchain-openai langchain langgraph-sdk langgraph-checkpoint-sqlite langsmith langchainhub==0.1.15 langchain-openai==0.1.3 langchain==0.1.16 langchain-core==0.1.42 pygraphviz==1.12 llama-index fastapi uvicorn openai

ERROR: Cannot install langchain-core, langchain-core==0.1.42, langchain-openai==0.1.3, langchain==0.1.16, langgraph==0.0.39, langgraph==0.0.40, langgraph==0.0.41, langgraph==0.0.42, langgraph==0.0.43, langgraph==0.0.44, langgraph==0.0.45, langgraph==0.0.46, langgraph==0.0.47, langgraph==0.0.48, langgraph==0.0.49, langgraph==0.0.50, langgraph==0.0.51, langgraph==0.0.52, langgraph==0.0.53, langgraph==0.0.54, langgraph==0.0.55, langgraph==0.0.56, langgraph==0.0.57, langgraph==0.0.58, langgraph==0.0.59, langgraph==0.0.60, langgraph==0.0.61, langgraph==0.0.62, langgraph==0.0.63, langgraph==0.0.64, langgraph==0.0.65, langgraph==0.0.66, langgraph==0.0.67, langgraph==0.0.68, langgraph==0.0.69, langgraph==0.1.1, langgraph==0.1.10, langgraph==0.1.11, langgraph==0.1.12, langgraph==0.1.13, langgraph==0.1.14, langgraph==0.1.15, langgraph==0.1.16, langgraph==0.1.17, langgraph==0.1.19, langgraph==0.1.2, langgraph==0.1.3, langgraph==0.1.4, langgraph==0.1.5, langgraph==0.1.6, langgraph==0.1.7, langgrap

# SETUP Enviornment

In [ ]:
import os.path
import logging
import sys

from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    load_index_from_storage,
)

# loading env variables 
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

# openai_api_key = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = "Open_AI_Key"
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o", temperature=0,
    max_tokens=None,
)
print(llm.invoke("you are gpt model only right which im using via a key?").content)

Yes, that's correct. I am a language model developed by OpenAI, and you are accessing my capabilities through an API key. This allows you to interact with me and use my features in various applications. If you have any questions or need assistance, feel free to ask!


# Version 1 No CBT/DBT Agents

In [33]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from pydantic import BaseModel, Field
from langgraph.graph.message import AnyMessage, add_messages
from typing import Literal, Annotated
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from typing import TypedDict, List
from openai import OpenAI
import os 
from langchain_openai import ChatOpenAI
import yaml
import csv

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

# Define the state
class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

openai_api_key = os.getenv("OPENAI_API_KEY")
BOT_USED = ""
# Load prompts from YAML
def load_prompts(file_path='prompts.yaml'):
    with open(file_path, 'r') as file:
        return yaml.safe_load(file)

prompts = load_prompts()

# Initialize OpenAI model
model = ChatOpenAI(model="gpt-4o", temperature=0.7)

# Define prompts for different agents
planner_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a planner agent that decides which specialized agent to call based on the user's input. Respond with one of 'suicide_prevention', 'conversational', 'anger_management', 'motivational', or 'mindfulness' based on the user's emotion."),
    ("human", "{input}"),
])

conversational_prompt = ChatPromptTemplate.from_messages([
    ("system", prompts['empathetic_agent_prompt']),
    ("human", "{input}"),
])

suicide_prevention_prompt = ChatPromptTemplate.from_messages([
    ("system", prompts['suicide_prevention_agent_prompt']),
    ("human", "{input}"),
])


anger_management_prompt = ChatPromptTemplate.from_messages([
    ("system", prompts['anger_prevention_agent_prompt']),
    ("human", "{input}"),
])

motivational_prompt = ChatPromptTemplate.from_messages([
    ("system", prompts['motivational_agent_prompt']),
    ("human", "{input}"),
])

mindfulness_prompt = ChatPromptTemplate.from_messages([
    ("system", prompts['mindfulness_agent_prompt']),
    ("human", "{input}"),
])

# Define router
def route_query(state: State):
    print(state)
    class RouteQuery(BaseModel):
        """Route a user query to the most relevant node."""
        route: Literal["conversational", "suicide_prevention", "anger_management", "motivational", "mindfulness"] = Field(
            ...,
            description="Choose the appropriate agent based on the user's emotions."
        )
    
    structured_llm_router = model.with_structured_output(RouteQuery)
    question_router = planner_prompt | structured_llm_router
    last_message = state["messages"][-1]
    resp = question_router.invoke({"input": last_message})
    return resp.route

# Define agent functions
def run_conversational_agent(state: State):
    global BOT_USED  # Declare BOT_USED as global
    print("Running conversational agent")
    convo_model = conversational_prompt | model
    response = convo_model.invoke(state["messages"])
    BOT_USED = "conversational_agent"  # Assign to global variable
    return {"messages": response}

def run_suicide_prevention_agent(state: State):
    global BOT_USED  # Declare BOT_USED as global
    print("Running suicide prevention agent")
    concern_model = suicide_prevention_prompt | model
    response = concern_model.invoke(state["messages"])
    BOT_USED = "suicide_agent"  # Assign to global variable
    return {"messages": response}

def run_anger_management_agent(state: State):
    global BOT_USED  # Declare BOT_USED as global
    print("Running anger management agent")
    anger_model = anger_management_prompt | model
    response = anger_model.invoke(state["messages"])
    BOT_USED = "anger_agent"  # Assign to global variable
    return {"messages": response}

def run_motivational_agent(state: State):
    global BOT_USED  # Declare BOT_USED as global
    print("Running motivational agent")
    motivation_model = motivational_prompt | model
    response = motivation_model.invoke(state["messages"])
    BOT_USED = "motivational_agent"  # Assign to global variable
    return {"messages": response}

def run_mindfulness_agent(state: State):
    global BOT_USED  # Declare BOT_USED as global
    print("Running mindfulness agent")
    mindfulness_model = mindfulness_prompt | model
    response = mindfulness_model.invoke(state["messages"])
    BOT_USED = "mindfulness_agent"  # Assign to global variable
    return {"messages": response}


# Create the graph
workflow = StateGraph(State)

# Add nodes for each agent
workflow.add_node("conversational", run_conversational_agent)
workflow.add_node("suicide_prevention", run_suicide_prevention_agent)
workflow.add_node("anger_management", run_anger_management_agent)
workflow.add_node("motivational", run_motivational_agent)
workflow.add_node("mindfulness", run_mindfulness_agent)

# Add edges
workflow.add_conditional_edges(
    START,
    route_query,
    {
        "conversational": "conversational",
        "suicide_prevention": "suicide_prevention",
        "anger_management": "anger_management",
        "motivational": "motivational",
        "mindfulness": "mindfulness"
    }
)
workflow.add_edge("conversational", END)
workflow.add_edge("suicide_prevention", END)
workflow.add_edge("anger_management", END)
workflow.add_edge("motivational", END)
workflow.add_edge("mindfulness", END)

# Compile the graph
memory = MemorySaver()
graph = workflow.compile(checkpointer=memory)

# Function to run a conversation turn
def chat(message: str, config: dict):
    result = graph.invoke({"messages": [HumanMessage(content=message)]}, config=config)
    return result

if __name__ == "__main__":
    config = {"configurable": {"thread_id": "1"}}
        
    # Open a CSV file for writing
    with open('chat_responses.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["User Input", "Triggered Agent", "Response"])  # Write the header

        # i = 0
        while True:
            user_input = input("You: ")
            if user_input.lower() in ["exit", "quit"]:
                print("Bot: Goodbye!")
                break
                
            response = chat(user_input, config)
            # response = response["messages"].content
            response = response['messages'][-1].content
            print(f"Sukoon: {response}")

            # Log the interaction to the CSV
            writer.writerow([user_input, BOT_USED, response])  # Write the data
            print(BOT_USED,"**********")
            BOT_USED = ""

{'messages': [HumanMessage(content='im feeling low', additional_kwargs={}, response_metadata={}, id='87d15a0c-66b0-4482-9baa-2f9176a94884')]}
Running suicide prevention agent
Sukoon: I'm really sorry to hear that you're feeling this way. Your feelings are important, and I'm here to listen. It's really important to talk to someone who can provide support. Your life matters, and let's get you the support you need right now.

Please consider reaching out to a professional who can help. You can contact one of these helplines for immediate support:
- iCALL: +919152987821
- NIMHANS: +918046110007 or 14416

They have trained professionals who can help you work through what you're experiencing. You are not alone, and there are people who care about you and want to help.
suicide_agent **********
Bot: Goodbye!


# Version 2 : DBT CBT added to above 5
## Mindfulness removed

In [20]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from pydantic import BaseModel, Field
from langgraph.graph.message import AnyMessage, add_messages
from typing import Literal, Annotated
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from typing import TypedDict, List
from openai import OpenAI
import os 
from langchain_openai import ChatOpenAI
import yaml
# from semantic_router import Route
# from transformers import pipeline
from typing import Dict

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

# Define the state
class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

openai_api_key = os.getenv("OPENAI_API_KEY")

# Load prompts from YAML
def load_prompts(file_path='prompts.yaml'):
    with open(file_path, 'r') as file:
        return yaml.safe_load(file)

prompts = load_prompts()

# Initialize OpenAI model
model = ChatOpenAI(model="gpt-4o", temperature=0.7)

# Define prompts for different agents
# planner_prompt = ChatPromptTemplate.from_messages([
#     ("system", "You are a planner agent that decides which specialized agent to call based on the user's input. Respond with one of 'suicide_prevention', 'conversational', 'anger_management', 'motivational', or 'mindfulness' based on the user's emotion."),
#     ("human", "{input}"),
# ])

planner_prompt = ChatPromptTemplate.from_messages([
    ("system", prompts['planner_agent_prompt']),
    ("human", "{input}"),
])

conversational_prompt = ChatPromptTemplate.from_messages([
    ("system", prompts['empathetic_agent_prompt']),
    ("human", "{input}"),
])

suicide_prevention_prompt = ChatPromptTemplate.from_messages([
    ("system", prompts['suicide_prevention_agent_prompt']),
    ("human", "{input}"),
])

anger_management_prompt = ChatPromptTemplate.from_messages([
    ("system", prompts['anger_prevention_agent_prompt']),
    ("human", "{input}"),
])

motivational_prompt = ChatPromptTemplate.from_messages([
    ("system", prompts['motivational_agent_prompt']),
    ("human", "{input}"),
])

# mindfulness_prompt = ChatPromptTemplate.from_messages([
#     ("system", prompts['mindfulness_agent_prompt']),
#     ("human", "{input}"),
# ])

dialectical_behavior_therapy_prompt = ChatPromptTemplate.from_messages([
    ("system", prompts['dbt_agent_prompt']),
    ("human", "{input}")
])

cognitive_behavioral_therapy_prompt = ChatPromptTemplate.from_messages([
    ("system", prompts['cbt_agent_prompt']),
    ("human", "{input}")
])

# Define router
def route_query(state: State):
  
    class RouteQuery(BaseModel):
        """Route a user query to the most relevant node based on the emotional or psychological state identified from the query intent."""
        
        route: Literal[
            "conversational", "suicide_prevention", "anger_management", 
            "motivational", "dialectical_behavior_therapy", "cognitive_behavioral_therapy"
        ] = Field(
            ...,
            description=(
                "Choose the most appropriate agent based on the user's emotional or psychological needs, inferred from their dialogue: "

                "'conversational' is ideal for users seeking general empathetic interaction, companionship, or simply wishing to engage in casual dialogue. This route aims to provide emotional support through open, non-directive conversation. \n"
                "Example: A user says, 'I've been feeling a bit lonely lately. I just need someone to talk to about my day.'\n"

                "'suicide_prevention' is critical for users who express thoughts of hopelessness, self-harm, suicidal ideation, or severe emotional distress. This route provides immediate intervention, offering resources and support to de-escalate the crisis. \n"
                "Example: A user states, 'I feel like no one would care if I were gone. I don't want to keep going anymore.'\n"

                "'anger_management' should be selected for users expressing frustration, irritability, or anger. This route helps the user manage their temper, process their emotions constructively, and reduce the risk of conflict escalation. \n"
                "Example: A user vents, 'I'm so mad at my boss! He keeps undermining me, and I'm about to explode.'\n"

                "'motivational' is suited for users who feel demotivated, struggle with low self-esteem, or are seeking encouragement to pursue their goals. This route offers positive reinforcement and practical strategies for improving self-worth and maintaining focus. \n"
                "Example: A user shares, 'I’ve been feeling stuck. Every time I try to work on my project, I lose motivation. What’s the point of even trying?' \n"

                "'dialectical_behavior_therapy' (DBT) should be used for users dealing with intense, fluctuating emotions or feeling emotionally overwhelmed. DBT teaches skills for emotional regulation, distress tolerance, and managing interpersonal relationships. \n"
                "Example: A user says, 'One moment I’m okay, but then I’m hit with this overwhelming sadness and anger. I don’t know how to control my emotions.'\n"

                "'cognitive_behavioral_therapy' (CBT) is appropriate for users struggling with negative or distorted thinking patterns, self-criticism, or irrational beliefs. CBT helps them reframe unhealthy thoughts into more positive, balanced perspectives. \n"
                "Example: A user confides, 'I always mess things up. No matter what I do, I feel like a failure, and it’s hard to think any differently.'"
            )
        )

    structured_llm_router = model.with_structured_output(RouteQuery)
    question_router = planner_prompt | structured_llm_router
    last_message = state["messages"][-1]
    resp = question_router.invoke({"input": last_message})
    return resp.route

# Define agent functions
def run_conversational_agent(state: State):
    print("Running conversational agent")
    convo_model = conversational_prompt | model
    response = convo_model.invoke(state["messages"])
    return {"messages": response}

def run_suicide_prevention_agent(state: State):
    print("Running suicide prevention agent")
    concern_model = suicide_prevention_prompt | model
    response = concern_model.invoke(state["messages"])
    return {"messages": response}

def run_anger_management_agent(state: State):
    print("Running anger management agent")
    anger_model = anger_management_prompt | model
    response = anger_model.invoke(state["messages"])
    return {"messages": response}

def run_motivational_agent(state: State):
    print("Running motivational agent")
    motivation_model = motivational_prompt | model
    response = motivation_model.invoke(state["messages"])
    return {"messages": response}

# def run_mindfulness_agent(state: State):
#     print("Running mindfulness agent")
#     mindfulness_model = mindfulness_prompt | model
#     response = mindfulness_model.invoke(state["messages"])
#     return {"messages": response}

def run_dialectical_behavior_therapy_agent(state: State):
    print("Running dialectical_behavior_therapy agent")
    dialectical_behavior_therapy_model = dialectical_behavior_therapy_prompt | model
    response = dialectical_behavior_therapy_model.invoke(state["messages"])
    return {"messages": response}

def run_cognitive_behavioral_therapy_agent(state: State):
    print("Running cognitive_behavioral_therapy agent")
    cognitive_behavioral_therapy_model = cognitive_behavioral_therapy_prompt | model
    response = cognitive_behavioral_therapy_model.invoke(state["messages"])
    return {"messages": response}

# Create the graph
workflow = StateGraph(State)

# Add nodes for each agent
workflow.add_node("conversational", run_conversational_agent)
workflow.add_node("suicide_prevention", run_suicide_prevention_agent)
workflow.add_node("anger_management", run_anger_management_agent)
workflow.add_node("motivational", run_motivational_agent)
# workflow.add_node("mindfulness", run_mindfulness_agent)
workflow.add_node("dialectical_behavior_therapy", run_dialectical_behavior_therapy_agent)
workflow.add_node("cognitive_behavioral_therapy", run_cognitive_behavioral_therapy_agent)

# Add edges
workflow.add_conditional_edges(
    START,
    route_query,
    {
        "conversational": "conversational",
        "suicide_prevention": "suicide_prevention",
        "anger_management": "anger_management",
        "motivational": "motivational",
        # "mindfulness": "mindfulness",
        "dialectical_behavior_therapy": "dialectical_behavior_therapy",
        "cognitive_behavioral_therapy": "cognitive_behavioral_therapy"
    }
)

workflow.add_edge("conversational", END)
workflow.add_edge("suicide_prevention", END)
workflow.add_edge("anger_management", END)
workflow.add_edge("motivational", END)
# workflow.add_edge("mindfulness", END)
workflow.add_edge("dialectical_behavior_therapy", END)
workflow.add_edge("cognitive_behavioral_therapy", END)

# Compile the graph
memory = MemorySaver()
graph = workflow.compile(checkpointer=memory)


# Function to run a conversation turn
def chat(message: str, config: dict):
    result = graph.invoke({"messages": [HumanMessage(content=message)]}, config=config)
    return result["messages"][-1]

if __name__ == "__main__":
    config = {"configurable": {"thread_id": "1"}}
    i = 0
    while True and i<5:
        i+=1
        user_input = input("You: ")
        print("user query : ",user_input)
        if user_input.lower() in ["exit", "quit"]:
            print("Bot: Goodbye!")
            break
        response = chat(user_input, config)
        print("memory : ",memory)
        print("Sukoon:", response.content)

user query :  hello my name is joel
Running conversational agent
memory :  <langgraph.checkpoint.memory.MemorySaver object at 0x00000164AD6C22D0>
Sukoon: I hear you, Joel. It's nice to meet you. How have you been feeling lately? Remember, it's okay to share whatever's on your mind. 😊
user query :  my names?
Running conversational agent
memory :  <langgraph.checkpoint.memory.MemorySaver object at 0x00000164AD6C22D0>
Sukoon: I hear you, Joel. It sounds like there might be a bit of confusion. It's completely okay to take your time with things. How can I support you today? 😊
user query :  exit
Bot: Goodbye!


In [19]:
print(dir(memory))
print(memory.__dir__)
print(memory.__dict__)
print(memory.__getstate__)
print(memory.__getattribute__)
print(memory.list)
# for message in memory.state.get("messages", []):
#     print(message)

['__abstractmethods__', '__aenter__', '__aexit__', '__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_is_protocol', 'aget', 'aget_tuple', 'alist', 'aput', 'aput_writes', 'config_specs', 'get', 'get_next_version', 'get_tuple', 'list', 'put', 'put_writes', 'serde', 'storage', 'writes']
<built-in method __dir__ of MemorySaver object at 0x00000164AD60E590>
{'serde': <langgraph.checkpoint.serde.jsonplus.JsonPlusSerializer object at 0x00000164ABD4DBD0>, 'storage': defaultdict(<function MemorySaver.__init__.<locals>.<lambda> at 0x00000164AD701800>, {'1': defaultd

Bot: Goodbye!
